<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/cookbooks/upstage/Solar-Full-Stack LLM-101/05_3_OracleDB.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Seoul.W dataset

In [1]:
# set parameters

file = open("info/api.txt", "r")
api_key = file.read()
file.close()

file = open("info/datapath.txt", "r")
data_path = file.read()
file.close()

file = open("info/resultspath.txt", "r")
results_path = file.read()
file.close()

In [2]:
from langchain_upstage import UpstageEmbeddings

# 쿼리 전용 임베딩 모델
#query_embeddings = UpstageEmbeddings(api_key=api_key, model="solar-embedding-1-large-query")

# 문장 전용 임베딩 모델
passage_embeddings = UpstageEmbeddings(api_key=api_key, model="solar-embedding-1-large-passage")

## 1. build DB

In [3]:
from langchain_upstage import UpstageLayoutAnalysisLoader
import os

UPSTAGE_API_KEY = api_key

layzer = UpstageLayoutAnalysisLoader(api_key=UPSTAGE_API_KEY,file_path=os.path.join(data_path, 'SeoulW.pdf'), output_type="text")

docs = layzer.load()  # or layzer.lazy_load()

/home/aix7101/anaconda3/envs/yoonnlp/lib/python3.10/site-packages/langchain_upstage/layout_analysis.py:135: UserWarning: UpstageLayoutAnalysisLoader is deprecated.Please use langchain_upstage.document_parse.UpstageDocumentParseLoader instead.
  warnings.warn(
/home/aix7101/anaconda3/envs/yoonnlp/lib/python3.10/site-packages/langchain_upstage/layout_analysis_parsers.py:160: UserWarning: UpstageLayoutAnalysisParser is deprecated.Please use langchain_upstage.document_parse_parsers.UpstageDocumentParseParser instead.
  warnings.warn(


## 2. Text Split

In [11]:
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)

chunk_size, chunk_overlap = 6000, 600

# 2. Split
text_splitter = RecursiveCharacterTextSplitter.from_language(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap, language=Language.HTML
)
splits = text_splitter.split_documents(docs)
print("Splits:", len(splits))

Splits: 3


In [12]:
print(splits[0])
len(splits[0].page_content)

page_content='서울여자대학교 학칙제 1 장 총 칙제1조 (목적)본 대학은 대한민국의 교육이념과 기독교 정신을 바탕으로 정보화 사회와 고도의 산업기술사회에 대비할 수 있는 학문과 기술을 습득하고, 지
도적 인격을 도야하여, 국가와 인류사회 발전에 공헌할 수 있는 지ᆞ덕ᆞ술을 갖춘 여성지도자를 양성하는데 그 목적이 있다.제2조 (명칭)본 대학교는 서울여자대학교라 부른다.제3조 (편제)① 본 대학교에 교양대학, 인문대학, 사회과학대학, 과학기술융합대학, 미래산업융합대학, 아트앤디자인스쿨, 자유전공학부, 글로벌통상학부,
대학원, 교육대학원, 휴먼서비스대학원을 둔다. 글로벌통상학부는 외국인 유학생을 대상으로 하는 정원외 전담학과로 운영한다. <개정 2024.
3. 27.>
② 대학원, 교육대학원, 휴먼서비스대학원의 학칙과 시행세칙은 따로 정한다.제4조 (학생의 정원)① 학생의 정원은 입학정원을 기준으로 하며 모집단위별 입학정원은 <별표1> 과 같다.
② 모집 단위별 전공결정에 관한 사항은 총장이 따로 정한다.제 2 장 부속기관 부속교육기관 부설연구기관제5조 (부속시설)① 대학교에는 다음의 부속시설을 둘 수 있다.1. 부속기관：도서관, 기숙사, 박물관, 학보사, 방송국, 대학교회, 에코캠퍼스추진사업단
2. 부속교육기관：정보보호 영재교육원, 한국생태학교
3. <삭제>
4. 부설연구기관：바롬인성교육연구소, 인문과학연구소, 사회과학연구소, 자연과학연구소, 미래산업융합연구소, 조형연구소, 여성연구소,
R&D연구소, 아동연구원
② 부속시설에 각각 장을 두며 부속시설의 장은 조교수 이상의 교원으로 겸보한다.
③ 부속시설의 장은 총장의 명을 받아 시설에 관한 사무를 관장하고 소속 교직원을 지휘, 감독한다.
④ 부속시설에 필요한 부서를 둘 수 있으며 부서장은 일반직 또는 교원으로 보하되 그 분장업무는 따로 규칙으로 정한다.
⑤ 제1항 제4호의 부설연구기관에 별도의 특정연구센터를 둔다. 특정연구센터의 설치, 운영에 관한 사항은 본 대학교 「부설연구소의 설치 및
운영에 관한 규

6000

In [13]:
chunk = []
for index in range(0,len(splits)) : # context 받아오기
    chunk.append(splits[index].page_content)

embedded_documents = passage_embeddings.embed_documents(chunk)

In [14]:
import numpy as np
np.save(data_path+f'topic_is/topic_is_ewha', np.array(chunk[0]))
np.save(data_path+f'topic_is/topic_is_ewha_embed', np.array(embedded_documents[0]))